# Dataset Cleaning

In [80]:
# useful libraries
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1. Uploading dataset [[source]](https://www.phase-trans.msm.cam.ac.uk/map/data/materials/welddb-b.html)

In [82]:
# # Path to the keys.json file (from /src/cleaning/ to /config)
# KEYS_PATH = os.path.join("..", "..", "config", "keys.json")


# with open(KEYS_PATH, "r", encoding="utf-8") as f:
#     keys = json.load(f)

# HF_TOKEN = keys.get("hf_token")

In [83]:
df = pd.read_parquet('../../data/welddb.parquet')
df

,Carbon concentration / (weight%),Silicon concentration / (weight%),Manganese concentration / (weight%),Sulphur concentration / (weight%),Phosphorus concentration / (weight%),Nickel concentration / (weight%),Chromium concentration / (weight%),Molybdenum concentration / (weight%),Vanadium concentration / (weight%),Copper concentration / (weight%),...,Charpy temperature / °C,Charpy impact toughness / J,Hardness / kgmm-2,50 % FATT,Primary ferrite in microstructure / %,Ferrite with second phase / %,Acicular ferrite / %,Martensite / %,Ferrite with carbide aggreagate / %,Weld ID
0,0.037,0.30,0.65,0.008,0.012,0,N,N,N,N,...,N,N,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Aaw
1,0.037,0.30,0.65,0.008,0.012,0,N,N,N,N,...,-28,100,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Aawch
2,0.037,0.30,0.65,0.008,0.012,0,N,N,N,N,...,-38,100,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Aht
3,0.037,0.31,1.03,0.007,0.014,0,N,N,N,N,...,N,N,N,N,N,N,N,N,N,Evans-Ni/CMn-1990/1991-0Baw
4,0.037,0.31,1.03,0.007,0.014,0,N,N,N,N,...,-48,100,N,N,32,28,40,0,0,Evans-Ni/CMn-1990/1991-0Bawch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647,0.100,0.35,0.90,0.008,0.016,0.60,8.6,0.98,0.18,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX35
1648,0.088,0.36,0.88,0.008,0.017,0.57,8.4,0.94,0.19,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX36
1649,0.090,0.34,0.89,0.008,0.016,0.17,8.2,0.94,0.02,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX37
1650,0.092,0.35,0.90,0.008,0.016,0.54,8.4,0.97,0.17,N,...,N,N,N,N,N,N,N,N,N,Birmingham-MAX38


In [84]:
print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

Dataset loaded: 1652 rows, 44 columns


In [85]:
df.columns

Index(['Carbon concentration / (weight%)', 'Silicon concentration / (weight%)',
       'Manganese concentration / (weight%)',
       'Sulphur concentration / (weight%)',
       'Phosphorus concentration / (weight%)',
       'Nickel concentration / (weight%)',
       'Chromium concentration / (weight%)',
       'Molybdenum concentration / (weight%)',
       'Vanadium concentration / (weight%)',
       'Copper concentration / (weight%)', 'Cobalt concentration / (weight%)',
       'Tungsten concentration / (weight%)',
       'Oxygen concentration / parts per million by weight',
       'Titanium concentration / parts per million by weight',
       'Nitrogen concentration / parts per million by weight',
       'Aluminium concentration / parts per million by weight',
       'Boron concentration / parts per million by weight',
       'Niobium concentration / parts per million by weight',
       'Tin concentration / parts per million by weight',
       'Arsenic concentration / parts per millio

## 2. Dataset Cleaning

### 2.1. General Overview

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 44 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Carbon concentration / (weight%)                       1652 non-null   float64
 1   Silicon concentration / (weight%)                      1652 non-null   float64
 2   Manganese concentration / (weight%)                    1652 non-null   float64
 3   Sulphur concentration / (weight%)                      1652 non-null   object 
 4   Phosphorus concentration / (weight%)                   1652 non-null   object 
 5   Nickel concentration / (weight%)                       1652 non-null   object 
 6   Chromium concentration / (weight%)                     1652 non-null   object 
 7   Molybdenum concentration / (weight%)                   1652 non-null   object 
 8   Vanadium concentration / (weight%)              

In [87]:
column_name_mapping = {
    "Carbon concentration / (weight%)": "carbon_wt_pct",
    "Silicon concentration / (weight%)": "silicon_wt_pct",
    "Manganese concentration / (weight%)": "manganese_wt_pct",
    "Sulphur concentration / (weight%)": "sulfur_wt_pct",
    "Phosphorus concentration / (weight%)": "phosphorus_wt_pct",
    "Nickel concentration / (weight%)": "nickel_wt_pct",
    "Chromium concentration / (weight%)": "chromium_wt_pct",
    "Molybdenum concentration / (weight%)": "molybdenum_wt_pct",
    "Vanadium concentration / (weight%)": "vanadium_wt_pct",
    "Copper concentration / (weight%)": "copper_wt_pct",
    "Cobalt concentration / (weight%)": "cobalt_wt_pct",
    "Tungsten concentration / (weight%)": "tungsten_wt_pct",
    "Oxygen concentration / parts per million by weight": "oxygen_ppm",
    "Titanium concentration / parts per million by weight": "titanium_ppm",
    "Nitrogen concentration / parts per million by weight": "nitrogen_ppm",
    "Aluminium concentration / parts per million by weight": "aluminium_ppm",
    "Boron concentration / parts per million by weight": "boron_ppm",
    "Niobium concentration / parts per million by weight": "niobium_ppm",
    "Tin concentration / parts per million by weight": "tin_ppm",
    "Arsenic concentration / parts per million by weight": "arsenic_ppm",
    "Antimony concentration / parts per million by weight": "antimony_ppm",
    "Current / A": "current_A",
    "Voltage / V": "voltage_V",
    "AC or DC": "current_type",
    "Electrode positive or negative": "electrode_polarity",
    "Heat input / kJmm-1": "heat_input_kJmm",
    "Interpass temperature / °C": "interpass_temp_C",
    "Type of weld;": "weld_type",
    "Post weld heat treatment temperature / °C": "pwht_temp_C",
    "Post weld heat treatment time / hours": "pwht_time_h",
    "Yield strength / MPa": "yield_strength_MPa",
    "Ultimate tensile strength / MPa": "uts_MPa",
    "Elongation / %": "elongation_pct",
    "Reduction of Area / %": "reduction_area_pct",
    "Charpy temperature / °C": "charpy_temp_C",
    "Charpy impact toughness / J": "charpy_toughness_J",
    "Hardness / kgmm-2": "hardness_kgmm2",
    "50 % FATT": "fatt50_C",
    "Primary ferrite in microstructure / %": "primary_ferrite_pct",
    "Ferrite with second phase / %": "ferrite_second_phase_pct",
    "Acicular ferrite / %": "acicular_ferrite_pct",
    "Martensite / %": "martensite_pct",
    "Ferrite with carbide aggreagate / %": "ferrite_carbide_pct",
    "Weld ID": "weld_id"
}


In [88]:
df = df.rename(columns=column_name_mapping)

In [89]:
categories = {
    "composition": [
        "carbon_wt_pct",
        "silicon_wt_pct",
        "manganese_wt_pct",
        "sulfur_wt_pct",
        "phosphorus_wt_pct",
        "nickel_wt_pct",
        "chromium_wt_pct",
        "molybdenum_wt_pct",
        "vanadium_wt_pct",
        "copper_wt_pct",
        "cobalt_wt_pct",
        "tungsten_wt_pct",
        "oxygen_ppm",
        "titanium_ppm",
        "nitrogen_ppm",
        "aluminium_ppm",
        "boron_ppm",
        "niobium_ppm",
        "tin_ppm",
        "arsenic_ppm",
        "antimony_ppm"
    ],
    
    "welding_parameters": [
        "current_A",
        "voltage_V",
        "current_type",
        "electrode_polarity",
        "heat_input_kJmm",
        "interpass_temp_C",
        "weld_type",
        "pwht_temp_C",
        "pwht_time_h"
    ],
    
    "mechanical_properties": [
        "yield_strength_MPa",
        "uts_MPa",
        "elongation_pct",
        "reduction_area_pct",
        "charpy_temp_C",
        "charpy_toughness_J",
        "hardness_kgmm2",
        "fatt50_C"
    ],
    
    "microstructure": [
        "primary_ferrite_pct",
        "ferrite_second_phase_pct",
        "acicular_ferrite_pct",
        "martensite_pct",
        "ferrite_carbide_pct"
    ],
    
    "identifiers": [
        "weld_id"
    ]
}


### 2.2. Initial Cleaning

#### 2.2.1. Nan values

We notice that 'N' is used to speicify Nan values => replace 'N' by `None`

In [90]:
df = df.replace('N', None)
print("Replaced 'N' with None for missing values")

Replaced 'N' with None for missing values


In [56]:
#df.describe(include='all').transpose()

We split columns into categories to simplify analysis later

#### 2.2.2. Numeric columns type into float64

In [57]:
#df.info()

In [91]:
non_numeric = [
    'weld_id', 'weld_type','current_type', 'electrode_polarity'
]

In [92]:
# Convert numeric columns to float
for col in df.columns:
    if col not in set(non_numeric):
        df[col] = (
            df[col].astype(str)                          # ensure string
            .str.replace(',', '.', regex=False)          # replace commas with dots
            .str.strip()                                 # strip spaces
        )
        df[col] = pd.to_numeric(df[col], errors='coerce')  # convert to float


In [93]:
print("Converted numeric columns to float64")

Converted numeric columns to float64


#### 2.2.3. Mapping categorical columns

In [61]:
# Option 1: Label Encoding (uncomment if preferred)
# type_weld_map = {
#     'MMA': 0, 'ShMA': 1, 'FCA': 2, 'SA': 3, 
#     'TSA': 4, 'SAA': 5, 'GTAA': 6, 'GMAA': 7, 
#     'NGSAW': 8, 'NGGMA': 9
# }
# ac_dc_map = {'AC': 0, 'DC': 1}
# electrode_map = {'+': 1, '0': 0, '-': -1}
# 
# df['Type of weld;'] = df['Type of weld;'].map(type_weld_map)
# df['AC or DC'] = df['AC or DC'].map(ac_dc_map)
# df['Electrode positive or negative'] = df['Electrode positive or negative'].map(electrode_map)

In [94]:
# Option 2: One-Hot Encoding

if 'weld_type' in df.columns:
    weld_type_dummies = pd.get_dummies(df['weld_type'], prefix='weld_type', dummy_na=False, dtype=int)
    df = pd.concat([df, weld_type_dummies], axis=1)
    df.drop('weld_type', axis=1, inplace=True)
    print(f"Created {len(weld_type_dummies.columns)} dummy variables for weld_type")

if 'current_type' in df.columns:
    ac_dc_dummies = pd.get_dummies(df['current_type'], prefix='current_type', dummy_na=False, dtype=int)
    df = pd.concat([df, ac_dc_dummies], axis=1)
    df.drop('current_type', axis=1, inplace=True)
    print(f"Created {len(ac_dc_dummies.columns)} dummy variables for current_type")

if 'electrode_polarity' in df.columns:
    electrode_dummies = pd.get_dummies(df['electrode_polarity'], prefix='electrode_polarity', dummy_na=False, dtype=int)
    df = pd.concat([df, electrode_dummies], axis=1)
    df.drop('electrode_polarity', axis=1, inplace=True)  
    print(f"Created {len(electrode_dummies.columns)} dummy variables for electrode_polarity")

Created 10 dummy variables for weld_type
Created 2 dummy variables for current_type
Created 3 dummy variables for electrode_polarity


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   carbon_wt_pct             1652 non-null   float64
 1   silicon_wt_pct            1652 non-null   float64
 2   manganese_wt_pct          1652 non-null   float64
 3   sulfur_wt_pct             1641 non-null   float64
 4   phosphorus_wt_pct         1642 non-null   float64
 5   nickel_wt_pct             697 non-null    float64
 6   chromium_wt_pct           784 non-null    float64
 7   molybdenum_wt_pct         791 non-null    float64
 8   vanadium_wt_pct           620 non-null    float64
 9   copper_wt_pct             564 non-null    float64
 10  cobalt_wt_pct             108 non-null    float64
 11  tungsten_wt_pct           63 non-null     float64
 12  oxygen_ppm                1256 non-null   float64
 13  titanium_ppm              865 non-null    float64
 14  nitrogen

## 3. Saving clean dataset

In [96]:
df = df.drop(columns=["weld_id"])

In [97]:
df.columns

Index(['carbon_wt_pct', 'silicon_wt_pct', 'manganese_wt_pct', 'sulfur_wt_pct',
       'phosphorus_wt_pct', 'nickel_wt_pct', 'chromium_wt_pct',
       'molybdenum_wt_pct', 'vanadium_wt_pct', 'copper_wt_pct',
       'cobalt_wt_pct', 'tungsten_wt_pct', 'oxygen_ppm', 'titanium_ppm',
       'nitrogen_ppm', 'aluminium_ppm', 'boron_ppm', 'niobium_ppm', 'tin_ppm',
       'arsenic_ppm', 'antimony_ppm', 'current_A', 'voltage_V',
       'heat_input_kJmm', 'interpass_temp_C', 'pwht_temp_C', 'pwht_time_h',
       'yield_strength_MPa', 'uts_MPa', 'elongation_pct', 'reduction_area_pct',
       'charpy_temp_C', 'charpy_toughness_J', 'hardness_kgmm2', 'fatt50_C',
       'primary_ferrite_pct', 'ferrite_second_phase_pct',
       'acicular_ferrite_pct', 'martensite_pct', 'ferrite_carbide_pct',
       'weld_type_FCA', 'weld_type_GMAA', 'weld_type_GTAA', 'weld_type_MMA',
       'weld_type_NGGMA', 'weld_type_NGSAW', 'weld_type_SA', 'weld_type_SAA',
       'weld_type_ShMA', 'weld_type_TSA', 'current_type_AC',

In [98]:
# Save data
output_path = "../../data/clean_weld_quality_dataset.csv"
df.to_csv(output_path, index=False)
print(f"Saved encoded dataset to: {output_path}")

Saved encoded dataset to: ../../data/clean_weld_quality_dataset.csv
